In [1]:
# all the necessary imports
import sys
sys.path.insert(0, '../scripts')
from readparquet import *
from pyspark.sql.functions import col
from pyspark.sql.functions import unix_timestamp

In [2]:
# let's import all the green taxi data for the past 6 months
sdf_g1 = read_parquet('../data/raw/green_tripdata_2021-11.parquet')
sdf_g2 = read_parquet('../data/raw/green_tripdata_2021-12.parquet')
sdf_g3 = read_parquet('../data/raw/green_tripdata_2022-01.parquet')
sdf_g4 = read_parquet('../data/raw/green_tripdata_2022-02.parquet')
sdf_g5 = read_parquet('../data/raw/green_tripdata_2022-03.parquet')
sdf_g6 = read_parquet('../data/raw/green_tripdata_2022-04.parquet')

22/08/01 17:21:20 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.207.1 instead (on interface eth1)
22/08/01 17:21:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/01 17:21:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/andrew/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/andrew/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# to see what features this data have, and how this data *roughly* looks like
df_g6 = sdf_g6.sample(0.01, seed=0).toPandas()
df_g6.info()
df_g6.sample(n=6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               731 non-null    int64         
 1   lpep_pickup_datetime   731 non-null    datetime64[ns]
 2   lpep_dropoff_datetime  731 non-null    datetime64[ns]
 3   store_and_fwd_flag     644 non-null    object        
 4   RatecodeID             644 non-null    float64       
 5   PULocationID           731 non-null    int64         
 6   DOLocationID           731 non-null    int64         
 7   passenger_count        644 non-null    float64       
 8   trip_distance          731 non-null    float64       
 9   fare_amount            731 non-null    float64       
 10  extra                  731 non-null    float64       
 11  mta_tax                731 non-null    float64       
 12  tip_amount             731 non-null    float64       
 13  tolls

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
626,1,2022-04-30 23:10:48,2022-04-30 23:25:14,N,1.0,181,65,1.0,0.00,19.20,0.0,0.5,0.00,0.0,NaN,0.3,20.00,1.0,1.0,0.0
102,2,2022-04-06 19:33:42,2022-04-06 19:39:17,N,1.0,134,134,5.0,0.97,5.50,0.0,0.5,0.94,0.0,NaN,0.3,7.24,1.0,1.0,0.0
320,1,2022-04-16 09:49:25,2022-04-16 09:49:45,N,5.0,80,80,1.0,0.00,35.00,0.0,0.0,0.00,0.0,NaN,0.0,35.00,1.0,2.0,0.0
335,2,2022-04-17 02:11:57,2022-04-17 02:16:12,N,1.0,7,146,1.0,1.17,5.50,0.0,0.5,0.00,0.0,NaN,0.3,6.30,2.0,1.0,0.0
398,2,2022-04-20 20:46:01,2022-04-20 20:50:53,N,1.0,75,75,1.0,0.48,5.00,0.0,0.5,0.00,0.0,NaN,0.3,5.80,2.0,1.0,0.0
682,2,2022-04-12 01:03:00,2022-04-12 01:24:00,None,NaN,196,129,NaN,2.31,13.11,0.0,0.0,1.49,0.0,NaN,0.3,14.90,NaN,NaN,NaN


In [2]:
# all the necessary features (I ditch the ones that I absolutely sure aren't needed, and keep the ones I'm 
# still somewhat unsure whether they are needed or not, and do some feature engineering later)
feats = ["trip_type", "trip_distance", "PULocationID", "DOLocationID", "RateCodeID", "fare_amount", "extra", \
    "tip_amount", "tolls_amount", "congestion_surcharge", "MTA_tax", "total_amount", "lpep_dropoff_datetime", "lpep_pickup_datetime"]
# improvement_surcharge is excluded because all rides charge the same $0.3.

sdf_g1 = sdf_g1.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_g2 = sdf_g2.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_g3 = sdf_g3.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_g4 = sdf_g4.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_g5 = sdf_g5.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_g6 = sdf_g6.select(col(feats[0]), col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), \
    col(feats[6]), col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col(feats[11]), col(feats[12]), col(feats[13]))

# I also need data for the length of each trip, and that is acquired by calculating
# the difference between lpep_pickup_datetime and lpep_dropoff_datetime.
time_diff = unix_timestamp("lpep_dropoff_datetime") - unix_timestamp("lpep_pickup_datetime")

sdf_g = sdf_g1.union(sdf_g2).union(sdf_g3).union(sdf_g4).union(sdf_g5).union(sdf_g6)
sdf_g = sdf_g.withColumn("trip_duration", time_diff)

# only need the trip duration, so these ones can be dropped off
sdf_g = sdf_g.drop("lpep_dropoff_datetime").drop("lpep_pickup_datetime")

NameError: name 'sdf_g1' is not defined

In [ ]:
# save this as a new parquet file
sdf_g.write.mode("overwrite").parquet("../data/curated/green")
# to copy what tute1 said: "Your directory might look a bit funky. Don't worry, just leave it as is,
# (we don't have time to cover everything about Spark unfortunately) and you can just read in the directory as is."
# I TAKE YOUR WORD FOR THIS, OKAY!!

Now, do the same thing (yes, same exact thing) for the yellow taxi data.

In [3]:
sdf_y1 = read_parquet('../data/raw/yellow_tripdata_2021-11.parquet')
sdf_y2 = read_parquet('../data/raw/yellow_tripdata_2021-12.parquet')
sdf_y3 = read_parquet('../data/raw/yellow_tripdata_2022-01.parquet')
sdf_y4 = read_parquet('../data/raw/yellow_tripdata_2022-02.parquet')
sdf_y5 = read_parquet('../data/raw/yellow_tripdata_2022-03.parquet')
sdf_y6 = read_parquet('../data/raw/yellow_tripdata_2022-04.parquet')
df_y6 = sdf_y6.sample(0.01, seed=0).toPandas()
df_y6.info()
df_y6.sample(n=6)

22/08/01 17:21:42 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.207.1 instead (on interface eth1)
22/08/01 17:21:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/01 17:21:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35795 entries, 0 to 35794
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               35795 non-null  int64         
 1   tpep_pickup_datetime   35795 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  35795 non-null  datetime64[ns]
 3   passenger_count        34612 non-null  float64       
 4   trip_distance          35795 non-null  float64       
 5   RatecodeID             34612 non-null  float64       
 6   store_and_fwd_flag     34612 non-null  object        
 7   PULocationID           35795 non-null  int64         
 8   DOLocationID           35795 non-null  int64         
 9   payment_type           35795 non-null  int64         
 10  fare_amount            35795 non-null  float64       
 11  extra                  35795 non-null  float64       
 12  mta_tax                35795 non-null  float64       
 13  t

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
13111,2,2022-04-12 20:35:28,2022-04-12 20:41:00,1.0,0.90,1.0,N,90,113,1,5.50,0.0,0.5,2.20,0.0,0.3,11.00,2.5,0.0
31488,1,2022-04-29 00:44:13,2022-04-29 00:55:33,0.0,1.30,1.0,N,239,142,1,9.00,2.5,0.5,2.70,0.0,0.3,15.00,2.5,0.0
31591,1,2022-04-29 02:57:53,2022-04-29 03:29:38,1.0,3.70,1.0,N,236,113,1,20.50,3.5,0.5,4.95,0.0,0.3,29.75,2.5,0.0
4233,2,2022-04-05 04:52:37,2022-04-05 04:56:20,3.0,1.12,1.0,N,162,137,1,5.00,1.0,0.5,2.32,0.0,0.3,11.62,2.5,0.0
13075,1,2022-04-12 20:11:43,2022-04-12 20:23:09,1.0,1.80,1.0,N,151,142,2,8.50,2.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
35099,2,2022-04-12 09:03:00,2022-04-12 09:12:00,NaN,2.36,NaN,None,238,48,0,10.79,0.0,0.5,2.00,0.0,0.3,16.09,NaN,NaN


In [4]:
# the yellow taxi data does not have "trip_type", but it has "airport_fee", 
# so the total number of columns actually ends up being the same anyway
feats[12] = "tpep_pickup_datetime"
feats[13] = "tpep_dropoff_datetime"
time_diff = unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
sdf_y1 = sdf_y1.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), col(feats[6]), \
    col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col("airport_fee"), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_y1 = sdf_y1.withColumn("trip_duration", time_diff)
sdf_y1 = sdf_y1.drop("tpep_dropoff_datetime").drop("tpep_pickup_datetime")

sdf_y2 = sdf_y2.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), col(feats[6]), \
    col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col("airport_fee"), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_y3 = sdf_y3.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), col(feats[6]), \
    col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col("airport_fee"), col(feats[11]), col(feats[12]), col(feats[13]))


['trip_type', 'trip_distance', 'PULocationID', 'DOLocationID', 'RateCodeID', 'fare_amount', 'extra', 'tip_amount', 'tolls_amount', 'congestion_surcharge', 'MTA_tax', 'total_amount', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']


22/08/01 17:23:13 ERROR Utils: Aborting task
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.parquet.io.api.Binary.fromReusedByteArray(Binary.java:344)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetWriteSupport.$anonfun$makeWriter$10(ParquetWriteSupport.scala:217)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetWriteSupport.$anonfun$makeWriter$10$adapted(ParquetWriteSupport.scala:212)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetWriteSupport$$Lambda$3181/1431369788.apply(Unknown Source)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetWriteSupport.$anonfun$writeFields$1(ParquetWriteSupport.scala:163)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetWriteSupport$$Lambda$3189/755744063.apply$mcV$sp(Unknown Source)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetWriteSupport.consumeField(ParquetWriteSupport.scala:486)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetWr

22/08/01 17:23:16 WARN FileOutputCommitter: Could not delete file:/mnt/c/Users/andre/Downloads/Unimelb/MAST30034 Applied Data Science/Project 1/GitHub/data/curated/yellow/_temporary/0/_temporary/attempt_20220801172202882859211111092495_0007_m_000035_49
22/08/01 17:23:16 WARN FileOutputCommitter: Could not delete file:/mnt/c/Users/andre/Downloads/Unimelb/MAST30034 Applied Data Science/Project 1/GitHub/data/curated/yellow/_temporary/0/_temporary/attempt_202208011722027309268148982802965_0007_m_000003_17
22/08/01 17:23:16 WARN FileOutputCommitter: Could not delete file:/mnt/c/Users/andre/Downloads/Unimelb/MAST30034 Applied Data Science/Project 1/GitHub/data/curated/yellow/_temporary/0/_temporary/attempt_2022080117220244287902655249111_0007_m_000043_57
22/08/01 17:23:16 ERROR FileFormatWriter: Job job_2022080117220244287902655249111_0007 aborted.
22/08/01 17:23:16 ERROR FileFormatWriter: Job job_202208011722027309268148982802965_0007 aborted.
22/08/01 17:23:16 ERROR FileFormatWriter: Job j

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/andrew/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/andrew/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/andrew/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/andrew/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, se

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
sdf_y4 = sdf_y4.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), col(feats[6]), \
    col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col("airport_fee"), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_y5 = sdf_y5.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), col(feats[6]), \
    col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col("airport_fee"), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_y6 = sdf_y6.select(col(feats[1]), col(feats[2]), col(feats[3]), col(feats[4]), col(feats[5]), col(feats[6]), \
    col(feats[7]), col(feats[8]), col(feats[9]), col(feats[10]), col("airport_fee"), col(feats[11]), col(feats[12]), col(feats[13]))
sdf_y = sdf_y1.union(sdf_y2).union(sdf_y3).union(sdf_y4).union(sdf_y5).union(sdf_y6)


sdf_y.write.mode("overwrite").parquet("../data/curated/yellow")

This is getting repetitive, but, I still need to do it again for HVFV vehicles. (RIP my storage space)

#### NOTE: I will not be using fhvhv data anymore. These chunks of code from here all the way until the bottom will be soon deleted.

In [13]:
sdf_fhv1 = read_parquet('../data/raw/fhvhv_tripdata_2021-11.parquet')
sdf_fhv2 = read_parquet('../data/raw/fhvhv_tripdata_2021-12.parquet')
sdf_fhv3 = read_parquet('../data/raw/fhvhv_tripdata_2022-01.parquet')
sdf_fhv4 = read_parquet('../data/raw/fhvhv_tripdata_2022-02.parquet')
sdf_fhv5 = read_parquet('../data/raw/fhvhv_tripdata_2022-03.parquet')
sdf_fhv6 = read_parquet('../data/raw/fhvhv_tripdata_2022-04.parquet')
df_fhv6 = sdf_fhv6.sample(0.001, seed=0).toPandas()
df_fhv6.info()
df_fhv6.sample(n=6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17762 entries, 0 to 17761
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     17762 non-null  object        
 1   dispatching_base_num  17762 non-null  object        
 2   originating_base_num  12989 non-null  object        
 3   request_datetime      17762 non-null  datetime64[ns]
 4   on_scene_datetime     12989 non-null  datetime64[ns]
 5   pickup_datetime       17762 non-null  datetime64[ns]
 6   dropoff_datetime      17762 non-null  datetime64[ns]
 7   PULocationID          17762 non-null  int64         
 8   DOLocationID          17762 non-null  int64         
 9   trip_miles            17762 non-null  float64       
 10  trip_time             17762 non-null  int64         
 11  base_passenger_fare   17762 non-null  float64       
 12  tolls                 17762 non-null  float64       
 13  bcf             

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
4592,HV0003,B03404,B03404,2022-04-09 03:41:29,2022-04-09 03:47:46,2022-04-09 03:48:29,2022-04-09 04:09:27,94,147,3.25,...,1.81,0.00,0.0,0.00,19.22,N,N,,N,N
13949,HV0003,B03404,B03404,2022-04-25 01:33:48,2022-04-25 01:38:58,2022-04-25 01:39:39,2022-04-25 01:59:36,164,66,5.97,...,2.45,2.75,0.0,3.36,17.48,N,N,,N,N
2458,HV0003,B03404,B03404,2022-04-05 09:02:59,2022-04-05 09:05:18,2022-04-05 09:06:46,2022-04-05 09:15:31,127,235,1.20,...,0.70,0.00,0.0,0.00,6.11,N,N,,N,N
2460,HV0005,B03406,None,2022-04-05 09:29:14,NaT,2022-04-05 09:36:28,2022-04-05 09:50:16,181,225,3.33,...,1.19,0.00,0.0,0.00,13.72,N,N,N,N,N
8176,HV0003,B03404,B03404,2022-04-15 03:43:05,2022-04-15 03:49:38,2022-04-15 03:49:43,2022-04-15 04:06:53,22,22,2.38,...,1.49,0.00,0.0,0.00,11.84,N,N,,N,N
12115,HV0003,B03404,B03404,2022-04-22 03:12:18,2022-04-22 03:19:29,2022-04-22 03:21:30,2022-04-22 03:39:36,162,230,1.19,...,1.27,2.75,0.0,5.00,11.39,N,N,,N,N


In [14]:
sdf_fhv1.where(sdf_fhv1.shared_match_flag != "N").where(sdf_fhv1.hvfhs_license_num != "HV0005")

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag


A note on shared rides: "shared_request_flag" always shows "N". I don't know if this is an error from whoever originated the data or it's the way it is.

"shared_match_flag", however, can show "Y" even if the request is "N". The interesting thing is, only Lyft (HV0005) can show "Y", while the rest of the license num always shows "N".

In [15]:
sdf_fhv1.where(sdf_fhv1.shared_match_flag != "N").where(sdf_fhv1.hvfhs_license_num == "HV0005")

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
HV0005,B03406,null,2021-11-01 12:40:37,null,2021-11-01 12:44:51,2021-11-01 13:09:46,79,235,12.78,1495,53.74,0.0,1.61,4.77,2.75,0.0,0.0,26.7,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 13:11:54,null,2021-11-01 13:14:08,2021-11-01 13:25:58,48,163,2.215,710,15.7,0.0,0.47,1.39,2.75,0.0,0.0,8.41,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 14:12:36,null,2021-11-01 14:17:09,2021-11-01 15:01:02,158,49,10.089,2658,121.24,0.0,3.64,10.76,2.75,0.0,0.0,47.53,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:09:31,null,2021-11-01 15:14:12,2021-11-01 15:35:05,162,265,6.742,1253,52.15,20.0,2.16,0.0,0.0,0.0,0.0,27.73,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:01:14,null,2021-11-01 15:06:33,2021-11-01 15:33:44,114,155,10.921,1631,89.63,2.17,2.75,8.15,2.75,0.0,0.0,40.09,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 14:54:38,null,2021-11-01 15:18:59,2021-11-01 15:40:15,148,262,6.458,1526,25.31,0.0,0.76,2.25,2.75,0.0,0.0,21.47,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:34:14,null,2021-11-01 15:37:54,2021-11-01 15:47:10,246,164,1.282,556,33.69,0.0,1.01,2.99,2.75,0.0,0.0,19.92,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:50:47,null,2021-11-01 15:53:34,2021-11-01 16:08:18,114,255,3.454,884,39.77,0.0,1.19,3.53,2.75,0.0,0.0,18.46,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:24:16,null,2021-11-01 15:32:17,2021-11-01 15:38:19,241,127,1.623,436,20.67,0.0,0.62,1.83,0.0,0.0,0.0,10.45,N,Y,N,N,N
HV0005,B03406,null,2021-11-01 15:09:35,null,2021-11-01 15:14:53,2021-11-01 15:32:53,198,33,5.688,1111,44.14,0.0,1.32,3.92,0.0,0.0,0.0,24.01,N,Y,N,N,N


I know I only showed about 1/6 of the data here, but I tested on all of them and the result is consistent. As a result, I will ignore the "shared ride" columns due to the inconsistency of the data. Since I ignore said columns, I will also not include "Lyft" as part of my curated data, since they are the ones who have differing values in terms of shared rides.

In [16]:
# Sadly, I can't merge all these together because my device can't handle the size limit.

feats_fhv = ["hvfhs_license_num", "trip_miles", "trip_time", "PULocationID", "DOLocationID", "base_passenger_fare", \
    "tolls", "bcf", "sales_tax", "congestion_surcharge", "airport_fee", "tips", "driver_pay"]

sdf_fhv1 = sdf_fhv1.where(sdf_fhv1.hvfhs_license_num != "HV0005")
sdf_fhv2 = sdf_fhv2.where(sdf_fhv2.hvfhs_license_num != "HV0005")
sdf_fhv3 = sdf_fhv3.where(sdf_fhv3.hvfhs_license_num != "HV0005")
sdf_fhv4 = sdf_fhv4.where(sdf_fhv4.hvfhs_license_num != "HV0005")
sdf_fhv5 = sdf_fhv5.where(sdf_fhv5.hvfhs_license_num != "HV0005")
sdf_fhv6 = sdf_fhv6.where(sdf_fhv6.hvfhs_license_num != "HV0005")

sdf_fhv1 = sdf_fhv1.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv2 = sdf_fhv2.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv3 = sdf_fhv3.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv4 = sdf_fhv4.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv5 = sdf_fhv5.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))
sdf_fhv6 = sdf_fhv6.select(col(feats_fhv[0]), col(feats_fhv[1]), col(feats_fhv[2]), col(feats_fhv[3]), col(feats_fhv[4]), col(feats_fhv[5]), \
    col(feats_fhv[6]), col(feats_fhv[7]), col(feats_fhv[8]), col(feats_fhv[9]), col(feats_fhv[10]), col(feats_fhv[11]), col(feats_fhv[12]))


In [17]:
sdf_fhv1.write.mode("overwrite").parquet("../data/curated/fhvhv1")
sdf_fhv2.write.mode("overwrite").parquet("../data/curated/fhvhv2")
sdf_fhv3.write.mode("overwrite").parquet("../data/curated/fhvhv3")
sdf_fhv4.write.mode("overwrite").parquet("../data/curated/fhvhv4")
sdf_fhv5.write.mode("overwrite").parquet("../data/curated/fhvhv5")
sdf_fhv6.write.mode("overwrite").parquet("../data/curated/fhvhv6")


yay! all done